In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/cs6471proj2/amazon
# from IPython.display import clear_output

Mounted at /content/drive
/content/drive/MyDrive/cs6471proj2/amazon


## example code for label

In [ ]:
import json
import re
import os

def load_file(path2file):
    file_handle = open(path2file, 'r')
    file = json.load(file_handle)
    file_handle.close()
    return file

def remove_multiple_spaces(string):
	return re.sub(r'\s+', ' ', string).strip()

def save_log(log_, path2save):
    file_handle = open('/'.join([path2save, 'complete_idx_label.json']), 'r')
    file = json.load(file_handle)
    file_handle.close()
    for key, item in log_.items():
        file[key] = item
    file_handle = open('/'.join([path2save, 'complete_idx_label.json']), 'w')
    file = json.dump(file, file_handle)
    file_handle.close()

    print('%d SAMPLES LABELED AND SAVED'%len(list(log_.keys())))

    return {}
    

# input name
while(1):
    input_name = remove_multiple_spaces(input('Input 1.Ruiqi, 2.Zexing, 3.Haoliang > '))
    if input_name in ['1', '2', '3']:
        break
    print('PLEASE ENTER 1, 2, 3')
# input_name = '3'
name_dict = {'1':'Ruiqi', '2':'Zexing', '3':'Haoliang'}
print('WECOME, ', name_dict[input_name])

# open folders
path2newfiles = 'purpose_classification'
save_path = '/'.join([path2newfiles, name_dict[input_name]])
if not os.path.exists(save_path):
    os.makedirs(save_path)

# load files
idx2data = load_file('/'.join([path2newfiles, 'global_country_idx2data.json']))
common_country_sample_idx = load_file('/'.join([path2newfiles, 'common_country_sample_idx.json']))
token = load_file('/'.join([path2newfiles, 'global_country_token.json']))
pos = load_file('/'.join([path2newfiles, 'global_country_data.json']))
ind_idx = load_file('/'.join([save_path, 'ind_idx.json']))
complete_idx_label = load_file('/'.join([save_path, 'complete_idx_label.json']))


# get uncomplete data
complete_idx = [int(key) for key, _ in complete_idx_label.items()]
print('NUMBER OF FINISHED LABEL:', len(complete_idx))
uncomplete_idx = [int(i) for i in ind_idx if int(i) not in complete_idx]
print('NUMBER OF LABEL LEFT:', len(uncomplete_idx))
current_log = {}
current_c = 0
label_candidates = {'1':"Origin", '2':"Attributes", '3':"Place of usage", '4':"Others", '5':"Ignore"}

# start giving questions
for global_idx in uncomplete_idx:
    current_c += 1
    #clear_output(wait=True)
    position = pos[str(global_idx)]
    review_idx = idx2data[str(global_idx)]
    sentence = token[review_idx['category']][str(review_idx['row_idx'])]
    print('NEW REVIEW')
    print('CATEGORY:', review_idx['category'])
    print('ENTIRE REVIEW: ', sentence[1])
    print('COUNTRY NAME: ', review_idx['country_name'])
    
#     from_pos = max(0, int(position['start'])-20)
#     end_pos = int(position['end']+20)
#     print('SURROUNDING WORDS:', ' '.join(sentence[0][from_pos: end_pos]))
    review_token = sentence[0]
    for i in range(int(position['end'])-int(position['start'])): 
        print(review_token[i+int(position['start'])])
        review_token[i+int(position['start'])] = '***'+review_token[i+int(position['start'])]+'***'
    from_pos = max(0, int(position['start'])-20)
    end_pos = int(position['end']+20)
    print('SURROUNDING WORDS:', ' '.join(review_token[from_pos: end_pos]))
    
    print('CHOICES:', label_candidates)
    while(1):
        input_label = input('Input > ')
        input_label = remove_multiple_spaces(input_label)

        # if it is a number case
        if input_label in label_candidates:
            print('SELECTED:', label_candidates[input_label])
            current_log[global_idx] = [int(input_label), label_candidates[input_label]]
            break

        # Check if it is save case
        elif input_label == 's' or input_label == 'save':
            if len(current_log) > 0:
                current_log = save_log(current_log, save_path)

        # Check if it is quit case
        elif input_label == 'q' or input_label == 'quit': break
        else:
            print('FAIL TO LABEL THE REVIEW')
    
    if (current_c+1)%10 == 0:
        current_log = save_log(current_log, save_path)

    if input_label == 'q' or input_label == 'quit':
        if len(list(current_log.keys()))>0:
            current_log = save_log(current_log, save_path)
        break
    

## load configurations

In [ ]:
country_worth2lookat = {'china':['china', 'ch'], 'us':['usa', 'us', 'america', 'united states', 'the states'], 
                      'uk':['uk', 'england', 'britain', 'ireland', 'scotland'], 
                      'japan':['japan'], 'mexico':['mexico'], 'germany': ['germany'], 
                      'canada':['canada'], 'india':['india'],  'australia':['australia'], 
                      'italy':['italy'], 'france':['france'], 'taiwan':['taiwan'], 'vietnam':['vietnam'], 
                      'thailand':['thailand'], 'brazil':['brazil'], 'korea':['korea'], 
                      'afghanistan':['afghanistan'], 'spain':['spain'], 'new zealand':['new zealand'], 
                      'singapore':['singapore'], 'iraq':['iraq'], 'russia':['russia'], 'switzerland':['switzerland']}


nationality_worth2lookat = {'american': ['american'], 'english': ['english'], 'german':['german'], 
                          'chinese':['chinese'], 'british':['british', 'scottish', 'irish'], 
                          'spanish':['spanish'], 'french':['french'], 'japanese':['japanese'], 
                          'european':['european'], 'australian':['australian'], 
                          'african':['african'], 'asian':['asian'], 'italian':['italian'], 
                          'canadian':['canadian'],  'indian':['indian'], 
                          'russian':['russian'], 'mexican':['mexican']} # 17

# build word2label
word2label_country = {}
word2label_nationality = {}
for key, item in country_worth2lookat.items():
    for word in item:
        word2label_country[word] = key
for key, item in nationality_worth2lookat.items():
    for word in item:
        word2label_nationality[word] = key

path2newfiles = 'purpose_classification'
file_handle = open('/'.join([path2newfiles, 'global_country_idx2data.json']), 'r')
idx2data = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'common_country_sample_idx.json']), 'r')
common_country_sample_idx = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'global_country_token.json']), 'r')
token = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'global_country_data.json']), 'r')
pos = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'global_country2idx.json']), 'r')
global_country2idx = json.load(file_handle)
file_handle.close()

## result analysis

In [ ]:
import copy
import json

# check percetage for each country

with open('purpose_classification/complete_idx_label_zexing4.json', 'r') as f:
    zexing = json.load(f)
with open('purpose_classification/complete_idx_label_ruiqi2.json', 'r') as f:
    ruiqi = json.load(f)

# idx2country:
idx2country = {}
for k, item in global_country2idx.items():
    for i in item:
        idx2country[int(i)] = k
    
print(len(zexing)+len(ruiqi))
total = copy.deepcopy(zexing)
for k, item in ruiqi.items():
    total[k] = item

print(len(total))

dict_ = {}
for k, item in total.items():
    if idx2country[int(k)] not in word2label_country:
        print(idx2country[int(k)])
        continue
    country = word2label_country[idx2country[int(k)]]
    if country not in dict_:
        dict_[country] = {item[0]:1}
    else:
        if item[0] not in dict_[country]:
            dict_[country][item[0]] = 1
        else:
            dict_[country][item[0]] += 1

print(dict_)

tem = 0
for c, di in dict_.items():

